# Bezier math

$Q(t) = \sum_{i=0}^{n} P_i B_{i, n} (t)$

Berstein polynomials: $B_{i, n}(t) = \binom{n}{i} t^i(1-t)^{n-i}$

## Cubic

n=3

$Q(t) = (1-t)^3 P_0 + 3 t(1-t)^2 P_1 + 3 t^2(1-t) P_2 + t^3 P_3$

$Q(t) = \begin{bmatrix}t^3 & t^2 & t & 1\end{bmatrix} \begin{bmatrix} -1 & 3 & -3 & 1 \\ 3 & -6 & 3 & 0 \\ -3 & 3 & 0 & 0 \\ 1 & 0 & 0 & 0 \end{bmatrix} \begin{bmatrix}P_0 \\ P_1 \\ P_2 \\ P_3 \end{bmatrix}$

## Quadratic

n=2

$Q(t) = (1-t)^2 P_0 + 2 t(1-t) P_1 + (1-t)^2 P_2$

$Q(t) = \begin{bmatrix}t^2 & t & 1\end{bmatrix} \begin{bmatrix} 1 & -2 & 1 \\ -2 & 2 & 0 \\ 1 & 0 & 0 \end{bmatrix} \begin{bmatrix}P_0 \\ P_1 \\ P_2 \end{bmatrix}$




In [1]:
%matplotlib widget

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

from myutils import eval, vec, curv, Plt

In [2]:
t, s, r, a, b, c, d = sp.symbols("t, s, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P_0 P_1 P_2 P_3')

# Equations

In [3]:
Bezier1 = lambda t, p0, p1: (1-t) * p0 + t * p1
Bezier2 = lambda t, p0, p1, p2: (1-t)**2 * p0 + 2 * t * (1-t) * p1 + t**2 * p2
Bezier3 = lambda t, p0, p1, p2, p3: (1-t)**3 * p0 + 3 * t * (1-t)**2 * p1 + 3 * t**2 * (1-t) * p2 + t**3 * p3

# Curves

In [13]:
plot = Plt(plt.figure(figsize=(6, 6)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plot.clr()

In [27]:
# points = (vec.vector(0, 0, 0), vec.vector(0, 1.0, 1), vec.vector(1, 1, 0))
# Q = Bezier2(t, *points) 

points = (vec.vector(0, 0, 0), vec.vector(0.5, 0, 1), vec.vector(0.5, 1, 1), vec.vector(1, 1, 0))
Q = Bezier3(t, *points) 

In [28]:
plot.points(points, 'k-', dashes=(1,1))
plot.curve(Q, 'c-', samples=10)

In [29]:
T, N, B = curv.frame(Q, t)
tt = np.linspace(0, 1, 8)
pnts = eval(Q, t=tt)
plot.vectors(pnts, eval(T, t=tt) * 0.1, color='r')
plot.vectors(pnts, eval(N, t=tt) * 0.1, color='g')
plot.vectors(pnts, eval(B, t=tt) * 0.1, color='b')

# Bezier3 -> Bezier2

Breaking b3 in 2 halves, fitting b2 to each half by its middle point: $\{p_0, p_m, p_2\} = \{b3(0), b3(0.25), b3(0.5)\}, \{b3(0.5), b3(0.75), b3(1.0)\}$

Closed form solution: $P_1 = 2 p_m - 0.5 p_0 - 0.5 p_2$

In [4]:
plot2 = Plt(plt.figure(figsize=(6, 6)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
points = (vec.vector(0, 0, 0), vec.vector(0.5, 0, 1), vec.vector(0.5, 1, 1), vec.vector(1, 1, 0))
Q0 = Bezier3(t, *points) 

In [6]:
def fit_b2_p1(q, pm):
    qm = q.subs({'t': 0.5}) # take middle point
    err = qm - pm
    err_sq = err.dot(err) # squared distance
    coords = ('P_1_x', 'P_1_y', 'P_1_z')  # coords to find
    err_sq_dp = [sp.diff(err_sq, coord) for coord in coords]
    solution = sp.linsolve(err_sq_dp, coords)
    return sp.Matrix(list(solution)[0] )

In [7]:
fit_b2_p1(Bezier2(t, P0, P1, P2), P)

Matrix([
[-0.5*P_0_x - 0.5*P_2_x + 2.0*P_x],
[-0.5*P_0_y - 0.5*P_2_y + 2.0*P_y],
[-0.5*P_0_z - 0.5*P_2_z + 2.0*P_z]])

In [8]:
pm0 = Q0.subs(t, 0.5)
pm1 = Q0.subs(t, 0.25)
pm2 = Q0.subs(t, 0.75)

Q1 = Bezier2(t, points[0], P1, pm0)
Q1 = vec.subs(Q1, {'P_1': fit_b2_p1(Q1, pm1)})
Q2 = Bezier2(t, pm0, P1, points[3])
Q2 = vec.subs(Q2, {'P_1': fit_b2_p1(Q2, pm2)})

In [9]:
plot2.curve(Q0, 'b-')
plot2.curve(Q1, 'r-')
plot2.curve(Q2, 'r-')
plot2.curve(Q1, 'ro', samples=6)
plot2.curve(Q2, 'ro', samples=6)

# B2 arclength approximation

Idea: $L_{line} < L_{arc} < L_{perimeter}$

$L_{arc} \approx \frac{2}{3}L_{line} + \frac{1}{3}L_{perimeter}$ 

[Raph Levien]



$p_t = q(t), p_1'(t) = (1-t) p_0 + t p_1$

$L_{line} = |p_t - p_0|, L_{perimeter} = |p_0 - p_1'| + |p_1' - p_t|$


$L_{arc} = \frac{2 |p_t - p_0| + |p_0 - p_1'| + |p_1' - p_t|}{3}$